In [3]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
from itertools import cycle
from sklearn.metrics import mean_squared_error
from fbprophet import Prophet
from tqdm import tqdm_notebook as tqdm
from lightgbm import LGBMRegressor




pd.set_option('max_columns', 50)
plt.style.use('bmh')

In [5]:
train = pd.read_csv("sales_train_validation_afcs2020.csv")
prices = pd.read_csv("sell_prices_afcs2020.csv")
calender = pd.read_csv("calendar_afcs2020.csv")
sample = pd.read_csv("sample_submission_afcs2020.csv")
evaluation = pd.read_csv("sales_train_evaluation_afcs2020.csv")



In [6]:
prices

,store_id,item_id,wm_yr_wk,sell_price
0,CA_3,HOBBIES_2_001,11105,5.47
1,CA_3,HOBBIES_2_001,11106,5.47
2,CA_3,HOBBIES_2_001,11107,5.47
3,CA_3,HOBBIES_2_001,11108,5.47
4,CA_3,HOBBIES_2_001,11109,5.47
...,...,...,...,...
32493,CA_3,HOBBIES_2_149,11617,0.97
32494,CA_3,HOBBIES_2_149,11618,0.97
32495,CA_3,HOBBIES_2_149,11619,0.97
32496,CA_3,HOBBIES_2_149,11620,0.97


In [4]:
prices['id'] =  prices.item_id.astype(str)  +"_"+ prices.store_id.astype(str)+ "_validation"

In [5]:
prices = prices.copy()

In [6]:
df = pd.melt(train, id_vars=['id'], var_name='d', value_name='sold').dropna()

In [24]:
df_test = pd.melt(evaluation, id_vars=['id'], var_name='d', value_name='sold').dropna()

In [7]:
df = pd.merge(df, calender, on='d', how='left')
df = pd.merge(df, prices, on ='id', how='left') 

In [25]:
df_test = pd.merge(df_test, calender, on='d', how='left')
df_test = pd.merge(df_test, prices, on ='id', how='left') 

In [8]:
df.head()

,id,d,sold,date,wm_yr_wk_x,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,store_id,item_id,wm_yr_wk_y,sell_price
0,HOBBIES_2_001_CA_3_validation,d_1,0,1/29/2011,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,CA_3,HOBBIES_2_001,11105,5.47
1,HOBBIES_2_001_CA_3_validation,d_1,0,1/29/2011,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,CA_3,HOBBIES_2_001,11106,5.47
2,HOBBIES_2_001_CA_3_validation,d_1,0,1/29/2011,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,CA_3,HOBBIES_2_001,11107,5.47
3,HOBBIES_2_001_CA_3_validation,d_1,0,1/29/2011,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,CA_3,HOBBIES_2_001,11108,5.47
4,HOBBIES_2_001_CA_3_validation,d_1,0,1/29/2011,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,CA_3,HOBBIES_2_001,11109,5.47


In [10]:
df2 = df.copy()

In [11]:
df2.head()

,id,d,sold,date,wm_yr_wk_x,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,store_id,item_id,wm_yr_wk_y,sell_price
0,HOBBIES_2_001_CA_3_validation,d_1,0,1/29/2011,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,CA_3,HOBBIES_2_001,11105,5.47
1,HOBBIES_2_001_CA_3_validation,d_1,0,1/29/2011,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,CA_3,HOBBIES_2_001,11106,5.47
2,HOBBIES_2_001_CA_3_validation,d_1,0,1/29/2011,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,CA_3,HOBBIES_2_001,11107,5.47
3,HOBBIES_2_001_CA_3_validation,d_1,0,1/29/2011,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,CA_3,HOBBIES_2_001,11108,5.47
4,HOBBIES_2_001_CA_3_validation,d_1,0,1/29/2011,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,CA_3,HOBBIES_2_001,11109,5.47


In [ ]:
# d_id = dict(zip(df2.id.cat.codes, df2.id))
# d_item_id = dict(zip(df.item_id.cat.codes, df.item_id))
# d_dept_id = dict(zip(df.dept_id.cat.codes, df.dept_id))
# d_cat_id = dict(zip(df.cat_id.cat.codes, df.cat_id))
# d_store_id = dict(zip(df.store_id.cat.codes, df.store_id))
# d_state_id = dict(zip(df.state_id.cat.codes, df.state_id))

In [12]:
df2.d = df2['d'].apply(lambda x: x.split('_')[1]).astype(np.int16)
# cols = df2.dtypes.index.tolist()
# types = df2.dtypes.values.tolist()
# for i,type in enumerate(types):
#     if type.name == 'category':
#         df2[cols[i]] = df2[cols[i]].cat.codes

In [26]:
df_test.d = df_test['d'].apply(lambda x: x.split('_')[1]).astype(np.int16)



In [32]:
d_item_id = dict(zip(df2.id.cat.codes, df2.id))


AttributeError: Can only use .cat accessor with a 'category' dtype

In [34]:
dummies_1 = pd.get_dummies(df2.id)

In [ ]:
df2[dummies_1.columns] =dummies_1

In [13]:
df3 = df2.copy()

In [14]:
df3 = df3[['d','sold','wday','month','year','id','sell_price']]

In [27]:
df_test_ = df_test[['d','sold','wday','month','year','id','sell_price']]

In [31]:
df_test_

,d,sold,wday,month,year,id,sell_price
0,1,0,1,1,2011,HOBBIES_2_001_CA_3_validation,5.47
1,1,0,1,1,2011,HOBBIES_2_001_CA_3_validation,5.47
2,1,0,1,1,2011,HOBBIES_2_001_CA_3_validation,5.47
3,1,0,1,1,2011,HOBBIES_2_001_CA_3_validation,5.47
4,1,0,1,1,2011,HOBBIES_2_001_CA_3_validation,5.47
...,...,...,...,...,...,...,...
63078613,1941,0,2,5,2016,HOBBIES_2_149_CA_3_validation,0.97
63078614,1941,0,2,5,2016,HOBBIES_2_149_CA_3_validation,0.97
63078615,1941,0,2,5,2016,HOBBIES_2_149_CA_3_validation,0.97
63078616,1941,0,2,5,2016,HOBBIES_2_149_CA_3_validation,0.97


In [18]:
#df3.to_pickle('pickled_data.pkl')

In [ ]:
valid = df_test_[(df3['d']>=1914) & (df_test_['d']<1942)][['id','d','sold']]
test = df_test_[df_test_['d']>=1942][['id','d','sold']]
valid_preds = evaluation['sold']

In [ ]:
predictions = []

In [17]:
products = df3.id.unique()

In [30]:
for product in products:
    df = df_test_[df_test_.id==product]
    
    #Split the data
    X_train, y_train = df_test_[df_test_['d']<1914].drop('sold',axis=1), df_test_[df_test_['d']<1914]['sold']
    X_train2, y_train2 = df_test_[df_test_['d']<1914].drop('sold',axis=1), df_test_[df_test_['d']<1942]['sold']


    
    X_valid, y_valid = df_test_[(df_test_['d']>=1914) & (df_test_['d']<1942)].drop('sold',axis=1), df_test_[(df_test_['d']>=1914) & (df_test_['d']<1942)]['sold']
    X_test = df_test_[df_test_['d']>=1942].drop('sold',axis=1)

    #Train and validate
    model = LGBMRegressor(
        n_estimators=1000,
        learning_rate=0.3,
        subsample=0.8,
        colsample_bytree=0.8,
        max_depth=8,
        num_leaves=50,
        min_child_weight=300
    )
    model.fit(X_train2, y_train2, eval_metric='rmse', verbose=20, early_stopping_rounds=20)
    predictions.append(model.predict(X_test))

    

ValueError: DataFrame.dtypes for data must be int, float or bool.
Did not expect the data types in the following fields: id